# Importing Libraries

In [ ]:
import gzip
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models, losses, optimizers

# Preprocessing

## Loading Data

In [ ]:
!wget https://github.com/SaadatChowdhury/BHaND/raw/master/Dataset/bhand.pkl.gz
f = gzip.open('/content/bhand.pkl.gz','rb')
trainSet,validSet,testSet = pickle.load(f, encoding = 'bytes')
f.close()

## Splitting into Datasets

In [ ]:
X_train, y_train = trainSet
X_val, y_val = validSet
X_test, y_test = testSet

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

## Data Visualization

In [ ]:
import matplotlib.pyplot as plt
img = np.resize(X_train[0], (32,32))
plt.imshow(img, cmap='gray')
plt.show()

## Reshaping

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
# n images, 32 x 32 dimension, 1 channel

X_train = X_train.reshape(-1, 32,32, 1)
X_val = X_val.reshape(-1, 32,32, 1)
X_test = X_test.reshape(-1, 32,32, 1)

X_train.shape, X_val.shape, X_test.shape

## One-Hot Encoding

In [ ]:
y_train.shape, y_val.shape, y_test.shape

In [ ]:
# 10 classes

y_train_oh = to_categorical(y_train)
y_val_oh = to_categorical(y_val)
y_test_oh = to_categorical(y_test)

y_train_oh.shape, y_val_oh.shape, y_test_oh.shape

# Model

## Setup

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation = 'softmax'))

model.summary()

## Training

In [ ]:
batch_size = 64
epochs = 10
num_classes = 10

In [ ]:
model.compile(loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(),metrics=['accuracy'])

training_history = model.fit(X_train, y_train_oh,
                             batch_size=batch_size,
                             epochs=epochs,
                             verbose=1,
                             validation_data=(X_val, y_val_oh))

## Evaluation

In [ ]:
test_eval = model.evaluate(X_test, y_test_oh, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = training_history.history['accuracy']
val_accuracy = training_history.history['val_accuracy']
loss = training_history.history['loss']
val_loss = training_history.history['val_loss']
epoch = range(len(accuracy))
plt.plot(epoch, accuracy, label='Training accuracy')
plt.plot(epoch, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epoch, loss, label='Training loss')
plt.plot(epoch, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Result Visualization

## Correct Classifications

In [ ]:
predicted_classes = model.predict(X_test)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
correct = np.where(predicted_classes==y_test)[0]

In [ ]:
print(f'correct labels - {len(correct)}')
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(32,32), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    plt.tight_layout()

## Incorrect Classifications

In [ ]:
correct = np.where(predicted_classes==y_test)[0]
print(f'Misclassified labels - {(y_test.shape[0]) - len(correct)}')
cnt = 0
for i in range (0,len(y_test)):
  if(predicted_classes[i]!=y_test[i]):
    
    plt.subplot(3,3,cnt+1)
    plt.imshow(X_test[i].reshape(32,32), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[i], y_test[i]))
    plt.tight_layout()
    cnt+=1
    if(cnt==9):
      break